# Yelp Data Challenge - NLP

BitTiger DS501

Jun 2017

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/Administrator/Desktop/Bittiger/501/Yelp Project/last_2_years_restaurant_reviews.csv')

In [3]:
df.head()

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,useful,user_id
0,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,ymlnR8UeFvB4FZL56tCZsA
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,9pSSL6X6lFpY3FCRLEH3og
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2016-08-22,1,1nK5w0VNfDlnR3bOz13dJQ,5,My husband and I went there for lunch on a Sat...,1,gm8nNoA3uB4In5o_Hxpq3g
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,CEtidlXNyQzgJSdF1ubPFw
4,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2015-02-02,0,_Uwp6FO1X-avE9wqTMC59w,5,This place is first class in every way. Lobste...,0,-Z7Nw2UF7NiBSAzfXNA_XA


### Define your feature variables, here is the text of the review

In [4]:
# Take the values of the column that contains review text data, save to a variable named "documents"
documents = df['text'].values

In [5]:
# inspect your documents, e.g. check the size, take a peek at elements of the numpy array
type(documents), documents.shape

(numpy.ndarray, (640718,))

### Define your target variable (any categorical variable that may be meaningful)

#### For example, I am interested in perfect (5 stars) and imperfect (1-4 stars) rating

In [7]:
# Make a column and take the values, save to a variable named "target"
df['perfect'] = df['stars']>4
target=df['perfect'].values

#### You may want to look at the statistic of the target variable

In [8]:
# To be implemented
target[:10]
target.mean()

0.4741461922405801

## Let's create training dataset and test dataset

In [9]:
from sklearn.cross_validation import train_test_split

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# Documents is your X, target is your y
# Now split the data to training set and test set

In [12]:
# Split to documents_train, documents_test, target_train, target_test
x_train, x_test, y_train, y_test = train_test_split(documents, target, test_size = .9, random_state=101
                                                   )

## Let's get NLP representation of the documents

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
# Create TfidfVectorizer, and name it vectorizer
vectorizer = TfidfVectorizer (stop_words = 'english', max_features = 1000)

In [16]:
# Train the model with your training data
x_train_ndarrays = vectorizer.fit_transform(x_train).toarray()
x_train_tfidf_matrix = vectorizer.fit_transform(x_train).todense()

In [17]:
# Get the vocab of your tfidf
words = vectorizer.get_feature_names()

In [18]:
# Use the trained model to transform your test data
x_test_ndarrays = vectorizer.transform(x_test).toarray()

## Similar review search engine

In [19]:
import numpy as np

# We will need these helper methods pretty soon

def get_top_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    '''
    return [labels[i] for i in np.argsort(lst)[::-1][:n]]  # np.argsort by default sorts values in ascending order

def get_bottom_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the lowest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["mouse", "rabbit"]
    '''
    pass  # To be implemented


In [20]:
# Let's use cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
# Draw an arbitrary review from test (unseen in training) documents
random_review_index = 16
query = documents[random_review_index]

In [23]:
# Transform the drawn review(s) to vector(s)
query_vectorized = vectorizer.transform([query]).todense()

In [24]:
# Calculate the similarity score(s) between vector(s) and training vectors
simiScores = []
for tf_doc in x_train_ndarrays:
    simiScores.append(cosine_similarity(query_vectorized.reshape(1,-1), tf_doc.reshape(1,-1)))

In [25]:
# Let's find top 5 similar reviews
n = 5
top5SimiReviews = get_top_values(np.array(simiScores).flatten(), n, x_train)

In [26]:
print('Our search query: %s' % query)

Our search query: Truly Fantastic!  Best Steak ever. Service was awesome and timely. They knew right when you needed something. The root beer float for dessert was great.


In [27]:
print('Most %s similar reviews:' % n)
for review in top5SimiReviews:
    print(review + '\n' + '-'*10)

Most 5 similar reviews:
Great $8 steak. Chicken fingers were awesome. The beer was good, although you could tell the draught lines needed to be cleaned. No problems with the service or using the coupon.
----------
Had amazing service from Lexi. She was one of the BEST waitresses I have ever had. She was truly great. The food was,great but the service was better. Thanks Lexi.
----------
Egg works is the best!!  Our server Veronica was awesome!  She was very attentive and always right there when we needed anything.  We'll see you next week...
----------
The food is fantastic.. I truly enjoy the bbq chicken wrap.... Ive since tried the signature salads.. Its fantastic... Love this place
----------
Great food and beer selection. It was exactly what we needed for our first meal after being on the road.
----------


#### Q: Does the result make sense to you?

A: The result makes sense to people since each review contain positive comments such as 'Great', 'amazing', 'best', 'fantastic', etc. 

## Classifying positive/negative review

#### Naive-Bayes Classifier

In [28]:
# Build a Naive-Bayes Classifier

from sklearn.naive_bayes import MultinomialNB

NBmodel = MultinomialNB()
NBmodel.fit(x_train_ndarrays, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
# Get score for training set
NBmodel.score(x_train_ndarrays, y_train)

0.7989105835713506

In [31]:
# Get score for test set
NBmodel.score(x_test_ndarrays, y_test)

0.7974063855356917

#### Logistic Regression Classifier

In [32]:
# Build a Logistic Regression Classifier

from sklearn.linear_model import LogisticRegression

logisModel = LogisticRegression()
logisModel.fit(x_train_ndarrays, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [33]:
# Get score for training set
logisModel.score(x_train_ndarrays, y_train)

0.8245696180799426

In [34]:
# Get score for test set
logisModel.score(x_test_ndarrays, y_test)

0.8166209136612174

#### Q: What are the key features(words) that make the positive prediction?

In [38]:
# Let's find it out by ranking
n = 20
get_top_values(logisModel.coef_.flatten(), n, words)

['amazing',
 'best',
 'thank',
 'perfection',
 'awesome',
 'fantastic',
 'gem',
 'delicious',
 'phenomenal',
 'perfect',
 'incredible',
 'highly',
 'great',
 'excellent',
 'outstanding',
 'favorite',
 'exceptional',
 'love',
 'glad',
 'bomb']

A: The coefficients are in the top 20 list, which have top 20 key features that make the positive prediction. 

#### Q: What are the key features(words) that make the negative prediction?

In [40]:
#?????
# Let's find it out by ranking
n = 20
get_bottom_values(logisModel.coef_.flatten(), n, words)

A: (insert your comments here)

#### Random Forest Classifier

In [41]:
# Build a Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier

RFmodel = RandomForestClassifier(n_estimators=20, max_depth=5, 
                                 min_samples_leaf = 20, random_state=1, n_jobs=-1)
RFmodel.fit(x_train_ndarrays, y_train)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [42]:
# Get score for training set
RFmodel.score(x_train_ndarrays, y_train)

0.7333270902592437

In [43]:
# Get score for test set
RFmodel.score(x_test_ndarrays, y_test)

0.7309931379162642

#### Q: What do you see from the training score and the test score?

A: Before we tunning the parameters, the model got overfitting issue, while after the tunning, the model become much better!

#### Q: Can you tell what features (words) are important by inspecting the RFC model?

In [44]:
n = 20
get_top_values(RFmodel.feature_importances_, n, words)

['amazing',
 'bad',
 'delicious',
 'best',
 'said',
 'wasn',
 'didn',
 'ok',
 'order',
 'vegas',
 'love',
 'better',
 'horrible',
 'awesome',
 'rude',
 'minutes',
 'waited',
 'disappointing',
 'favorite',
 'recommend']

## Extra Credit #1: Use cross validation to evaluate your classifiers

[sklearn cross validation](http://scikit-learn.org/stable/modules/cross_validation.html)

In [46]:
# To be implemented
from sklearn.model_selection import cross_val_score
scores = cross_val_score(RFmodel, x_train_ndarrays, y_train, 
                         cv=5, n_jobs = -1)
scores.mean()


0.7349502875591897

## Extra Credit #2: Use grid search to find best predictable classifier


[sklearn grid search tutorial (with cross validation)](http://scikit-learn.org/stable/modules/grid_search.html#grid-search)

[sklearn grid search documentation (with cross validation)](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)

In [ ]:
# To be implemented
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
LR = LogisticRegression()
param_grid_RF = {'penalty': ['l1', 'l2'],'C': [0.1, 1, 10, 100]}
grid_LR = GridSearchCV(LR, param_grid_RF, cv=5, scoring='accuracy', n_jobs=-1)
grid_LR_fit = grid_LR.fit(x_train_ndarrays, y_train)

clf_RF_final = grid_LR_fit.best_estimator_clf_RF_final.fit(x_train_ndarrays, y_train)

train_score = clf_RF_final.score(x_train_ndarrays, y_train)
test_score = clf_RF_final.score(x_test_ndarrays, y_test)

print("The final logistic model: \n %s"%clf_RF_final + '\n' +
      "The best trainging and test scores: %.6f and %.6f"%(train_score, test_score))